In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# from __future__ import division
import numpy as np
# import scipy.signal as sg
# from scipy.fftpack import rfft, fftfreq
from scipy.io import wavfile
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# %matplotlib inline
# import matplotlib.colors as colors
# from pydub import AudioSegment
import math as m
import tensorflow as tf
import utilities as util

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def gather_samples():
    '''
    Gather all of the files used to train the model
    Trombone and Guitar to setup a binary network
    '''
    sample_files = []
    location = '../samples/'
    instruments = ['Trombone', 'Guitar']
    for name in instruments:
        for note in (util.note_names('C1', 'C2')):
            for j in range(1, 6):
                file_name = location +name+"/"+name +"_"+note[:-1]+"("+str(j)+").wav"
                sample_files.append(file_name)
    return sample_files

In [3]:
nps_uniq, dfreqs, notes = util.precompute_spect()
smallest_nps = 44100/24 * 4/3 
smallest_nps = 2**int(np.log2(smallest_nps))
note_freq = util.frequencies('A0', 'D8')
sample_files = gather_samples()

In [4]:
def tf_normal_peak(vals, dfreq):
    '''
    Calculate the normal peak of the frequencies with tensors
    '''
    #print('vals correct')
    sums = tf.reduce_sum(vals, 0, keepdims=True)
    #print('sums calculated')
    weights = vals/sums
    #print('weights calculated')
    freqs = tf.range(tf.cast(tf.shape(vals)[0], 'float64'), dtype='float64')
    #print('freqs calculated')
    mean = tf.reduce_sum(
        tf.cast(weights, 'float64') * tf.cast(freqs[:,None], 'float64'),  
        axis=0,
        keepdims=True)
    #print('mean calculated')
    var = tf.truediv(tf.reduce_sum(tf.cast(((freqs[:,None]-mean)**2), 'float64')*tf.cast(weights, 'float64'), 0),dfreq)
    #print('var calculated')
    denom_var = tf.multiply(tf.constant(2.0, dtype='float64'), m.pi)
    #print('denominator variable calculated')
    denom_var_mult = tf.scalar_mul(denom_var, var)
    #print('multiplied denominator calculated')
    denom = tf.sqrt(tf.abs(denom_var_mult))
    #print('denominator calculated')
    dst_peak = tf.truediv(tf.constant(1.0, dtype='float64'),denom)
    #print('distribution peak calculated')
    peak = (tf.cast(dst_peak, 'float64') * tf.cast(dfreq, 'float64') * tf.cast(tf.squeeze(sums), 'float64')) if (dst_peak != 'Nan') else 0
    #print('peak calculated')
    return peak

In [5]:
def preprocess(filename):
    # Gathering the Data
    # Read the Wave File
    fs, wav = wavfile.read(filename)
    if wav.ndim == 2: wav = wav[:,0]
    wav = wav.astype(np.double)
    # Calculate the Spectrogram
    spectrogram = tf_spect(wav)
#     arg_sort = np.argsort(spectrogram,  axis=0)
    return spectrogram

In [6]:
def tf_spect(wav):   
    data = []
    first = True
    # Caclculating the Spectrogram
    for nperseg in nps_uniq:
        signals = tf.convert_to_tensor(wav, tf.float32)
#        signals = tf.placeholder(...)
#        frames = tf.contrib.signal.frame(signals, frame_length=np.int32(nperseg),
#                                         frame_step=np.int32(nperseg-smallest_nps))
        stfts = tf.contrib.signal.stft(signals, frame_length=np.int32(3*nperseg - 2*smallest_nps),
                                       frame_step=np.int32(nperseg),
                                       fft_length=np.int32(3*nperseg - 2*smallest_nps))
        #   power_spectrograms = tf.real(stfts * tf.conj(stfts))
        magnitude_spectrograms = tf.abs(stfts)
        data.append(magnitude_spectrograms)
        
        #if first:
        #    with tf.Session() as sess:
        #        out = sess.run(magnitude_spectrograms)
        #        print(out.shape)
        #    first = False
        
    # Empty new spectrogram
    full_spec = tf.Variable(tf.zeros((len(note_freq)-1, tf.shape(data[0][1])[0]), tf.float64))
    
    # Calculate the normal peaks in order to make a more clear spectrogram
    for i, (nps_ind, slc) in enumerate(notes):
        tf_spec = data[nps_ind]
        spec_slc = tf_spec[slc[0]:slc[1],:]
        tf_np = tf.cast(spec_slc, tf.float64) if slc[1]-slc[0] == 1 else tf_normal_peak(spec_slc, dfreqs[nps_ind])
        time = tf.shape(tf_spec)[0]
        full_spec[i,:tf.size(time)].assign(tf_np)
    
    # Return the note frequencies, original data, and spectrogram with normalized peaks
    return full_spec

In [35]:
wavfile.read??

In [42]:
def read(filename, mmap=False):
    """
    Return the sample rate (in samples/sec) and data from a WAV file
    Parameters
    ----------
    filename : string or open file handle
        Input wav file.
    mmap : bool, optional
        Whether to read data as memory mapped.
        Only to be used on real files (Default: False)
        .. versionadded:: 0.12.0
    Returns
    -------
    rate : int
        Sample rate of wav file
    data : numpy array
        Data read from wav file
    Notes
    -----
    * The file can be an open file or a filename.
    * The returned sample rate is a Python integer
    * The data is returned as a numpy array with a
      data-type determined from the file.
    """
    from scipy.io.wavfile import WAVE_FORMAT_PCM, WavFileWarning, _read_riff_chunk, _read_fmt_chunk, _skip_unknown_chunk, _read_data_chunk
    import warnings
    if hasattr(filename, 'read'):
        fid = filename
        mmap = False
    else:
        fid = open(filename, 'rb')

    try:
        file_size, is_big_endian = _read_riff_chunk(fid)
        fmt_chunk_received = False
        channels = 1
        bit_depth = 8
        format_tag = WAVE_FORMAT_PCM
        while fid.tell() < file_size:
            # read the next chunk
            chunk_id = fid.read(4)
            print(chunk_id)
            if not chunk_id:
                raise ValueError("Unexpected end of file.")
            elif len(chunk_id) < 4:
                raise ValueError("Incomplete wav chunk.")

            if chunk_id == b'fmt ':
                fmt_chunk_received = True
                fmt_chunk = _read_fmt_chunk(fid, is_big_endian)
                format_tag, channels, fs = fmt_chunk[1:4]
                bit_depth = fmt_chunk[6]
                if bit_depth not in (8, 16, 32, 64, 96, 128):
                    raise ValueError("Unsupported bit depth: the wav file "
                                     "has {}-bit data.".format(bit_depth))
            elif chunk_id == b'fact':
                _skip_unknown_chunk(fid, is_big_endian)
            elif chunk_id == b'data':
                if not fmt_chunk_received:
                    raise ValueError("No fmt chunk before data")
                data = _read_data_chunk(fid, format_tag, channels, bit_depth,
                                        is_big_endian, mmap)
            elif chunk_id == b'LIST':
                # Someday this could be handled properly but for now skip it
                _skip_unknown_chunk(fid, is_big_endian)
            elif chunk_id in (b'JUNK', b'Fake'):
                # Skip alignment chunks without warning
                _skip_unknown_chunk(fid, is_big_endian)
            else:
                warnings.warn("Chunk (non-data) not understood, skipping it.",
                              WavFileWarning)
                _skip_unknown_chunk(fid, is_big_endian)
    finally:
        if not hasattr(filename, 'read'):
            fid.close()
        else:
            fid.seek(0)

    return fs, data


In [43]:
read('../samples/Trombone/Trombone_A#(1).wav')

b'fmt '
b'data'


(44100,
 array([-25229, -24348, -23378, ...,      0,      0,      0], dtype=int16))

In [45]:
read('../samples/Guitar/Guitar_A#(1).wav')[1][:,0]

b'JUNK'
b'fmt '
b'data'
b'LGWV'
b'cue '
b'LIST'


/Users/Peeke/Honors/.env/lib/python3.6/site-packages/ipykernel_launcher.py:71: WavFileWarning: Chunk (non-data) not understood, skipping it.


array([ 30, -56,  17, ...,   0,   0,   0], dtype=int16)

In [7]:
s = preprocess('../samples/Trombone/Trombone_A#(1).wav')
# Sliced assignment is only supported for variables

In [9]:
# Preprocess all of the sample files
# Create lists of all of the frequencies, times and spectrograms
all_freqs = note_freq[:-1]
all_specs = []
for sf in sample_files:
    s = preprocess(sf)
    all_specs.append(s)


In [8]:
import warnings
warnings.filterwarnings("ignore", category=wavfile.WavFileWarning)

In [13]:
guitar_names = []
location = '../samples/'
instrument = 'Guitar'
for note in (util.note_names('A1', 'A2')):
    for j in range(1, 6):
        file_name = location +instrument+"/"+instrument +"_"+note[:-1]+"("+str(j)+").wav"
        guitar_names.append(file_name)


In [50]:
preprocess('../samples/Piano/Piano_A(1).wav')

<tf.Variable 'Variable_70:0' shape=(88, 1025) dtype=float64_ref>

In [20]:
preprocess('../samples/Trombone/Trombone_C(1).wav')

<tf.Variable 'Variable_68:0' shape=(88, 1025) dtype=float64_ref>

In [21]:
len(sample_files)

120

In [23]:
len(all_specs)

60

In [10]:
# Create a Network (1)
# x = tf.placeholder(shape=[None, 2], dtype=tf.float32)
x = tf.stack(all_specs)
nn = tf.layers.dense(x, 3)#, activation=tf.nn.sigmoid)
nn = tf.layers.dense(nn, 5)#, activation=tf.nn.sigmoid)
encoded = tf.layers.dense(nn, 2)#, activation=tf.nn.sigmoid)
ret = tf.layers.flatten(encoded)

In [11]:
with tf.Session() as sess:
#     for spectro in all_specs:
    sess.run(encoded)
    #for step in range(10000):
        #sess.run(ret, feed_dict={x:s})
        # sess.run(ret, feed_dict={x:tf.Variable(all_specs, dtype=tf.float64)})
        # sess.run(ret, feed_dict={x:tf.convert_to_tensor(all_specs, dtype=tf.float64)})
    #    if step % 5 == 0:
    #        print("step: {}, value: {}".format(step, val))


FailedPreconditionError: Attempting to use uninitialized value Variable_4
	 [[Node: Variable_4/read = Identity[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_4)]]

Caused by op 'Variable_4/read', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/Peeke/Honors/.env/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/Peeke/Honors/.env/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/Peeke/Honors/.env/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/Users/Peeke/Honors/.env/lib/python3.6/site-packages/tornado/platform/as

In [2]:
x

NameError: name 'x' is not defined

In [75]:
all_specs[0]

<tf.Variable 'Variable_158:0' shape=(88, 1025) dtype=float64_ref>

In [34]:
# Create a Convolutional Network (2)
x = tf.placeholder(shape=[None, None], dtype=tf.float32)
nn = tf.layers.conv2d(x, 3, 3)#, activation=tf.nn.sigmoid)
encoded2 = tf.layers.flatten(nn, 2)#, activation=tf.nn.sigmoid)

ValueError: Input 0 of layer conv2d_8 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, None]

In [27]:
import keras

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)


ModuleNotFoundError: No module named 'keras'